In [1]:
import albumentations as A
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

2026-02-17 15:38:16.516820: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-17 15:38:16.548021: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-17 15:38:17.505457: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
IMG_SIZE = 256
BATCH_SIZE = 8
EPOCHS = 25

BASE_PATH = "../archive/brisc2025/segmentation_task"

TRAIN_IMG = f"{BASE_PATH}/train/images"
TRAIN_MASK = f"{BASE_PATH}/train/masks"

TEST_IMG = f"{BASE_PATH}/test/images"
TEST_MASK = f"{BASE_PATH}/test/masks"

PRED_BASE = f"{BASE_PATH}/predictions/test"

PRED_MASK_DIR = f"{PRED_BASE}/masks"
OVERLAY_DIR = f"{PRED_BASE}/overlays"

os.makedirs(PRED_MASK_DIR, exist_ok=True)
os.makedirs(OVERLAY_DIR, exist_ok=True)

In [3]:
def load_image_mask(img_path, mask_path):

    #Load image 
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = img_to_array(img).astype("float32") / 255.0

    #Load mask (grayscale)
    mask = load_img(
        mask_path,
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="grayscale"
    )
    mask = img_to_array(mask).astype("float32") / 255.0

    #Binarize mask 
    mask = (mask > 0.5).astype("float32")

    return img, mask


In [4]:
def data_generator(img_dir, mask_dir, batch_size):

    img_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    while True:
        np.random.shuffle(img_files)

        for i in range(0, len(img_files), batch_size):

            batch_files = img_files[i:i + batch_size]
            imgs, masks = [], []

            for img_file in batch_files:

                img_path = os.path.join(img_dir, img_file)

                # mask name must match image stem
                mask_file = os.path.splitext(img_file)[0] + ".png"
                mask_path = os.path.join(mask_dir, mask_file)

                if not os.path.exists(mask_path):
                    print(f"Missing mask: {mask_file}")
                    continue

                img, mask = load_image_mask(img_path, mask_path)

                imgs.append(img)
                masks.append(mask)

            if len(imgs) > 0:
                yield np.array(imgs), np.array(masks)


In [5]:
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    
    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    return x

In [6]:
def attention_block(g, x, filters):
    g1 = layers.Conv2D(filters, 1, padding='same')(g)
    x1 = layers.Conv2D(filters, 1, padding='same')(x)
    psi = layers.Add()([g1, x1])
    psi = layers.Activation('relu')(psi)
    psi = layers.Conv2D(1, 1, padding='same', activation='sigmoid')(psi)
    return layers.Multiply()([x, psi])

def unet(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D()(c1)
    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D()(c2)
    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D()(c3)

    # Bottleneck
    bn = conv_block(p3, 512)

    # Decoder with Attention
    u4 = layers.Conv2DTranspose(256, 2, strides=2, padding='same')(bn)
    attn4 = attention_block(u4, c3, 256)
    u4 = layers.Concatenate()([u4, attn4])
    c4 = conv_block(u4, 256)

    u5 = layers.Conv2DTranspose(128, 2, strides=2, padding='same')(c4)
    attn5 = attention_block(u5, c2, 128)
    u5 = layers.Concatenate()([u5, attn5])
    c5 = conv_block(u5, 128)

    u6 = layers.Conv2DTranspose(64, 2, strides=2, padding='same')(c5)
    attn6 = attention_block(u6, c1, 64)
    u6 = layers.Concatenate()([u6, attn6])
    c6 = conv_block(u6, 64)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c6)
    return models.Model(inputs, outputs)


In [7]:
import tensorflow as tf

def dice_coefficient(y_true, y_pred):
    smooth = 1e-6

    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f)

    return (2. * intersection + smooth) / (
        tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth
    )


def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

In [8]:
model = unet()

model.compile(
    optimizer=Adam(1e-4),
    loss=bce_dice_loss,
    metrics=["accuracy", dice_coefficient]
)

model.summary()

I0000 00:00:1771322900.505581   14933 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4132 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 7,881,988 (30.07 MB)

 Trainable params: 7,876,356 (30.05 MB)

 Non-trainable params: 5,632 (22.00 KB)

In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs visible to TF:", tf.config.list_physical_devices("GPU"))


2026-02-16 20:56:08.462525: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 20:56:08.506078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-16 20:56:09.654834: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow version: 2.20.0
GPUs visible to TF: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

from sklearn.model_selection import train_test_split

# Split TRAIN into Train / Val

all_train_files = sorted([
    f for f in os.listdir(TRAIN_IMG)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

train_files, val_files = train_test_split(
    all_train_files,
    test_size=0.2,
    random_state=42
)

print("Train samples:", len(train_files))
print("Val samples:", len(val_files))


# Generators from subsets


# Define Augmentation Pipeline
# Define Augmentation Pipeline (Enhanced)
# Define Augmentation Pipeline (Enhanced)
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5),
    ], p=0.3),
    A.OneOf([
        A.CLAHE(clip_limit=2),
        A.RandomBrightnessContrast(),
        A.RandomGamma(),
    ], p=0.3),
    A.GaussNoise(p=0.2),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.2),
])

def subset_generator(img_dir, mask_dir, files, batch_size, transforms=None):

    while True:
        np.random.shuffle(files)

        for i in range(0, len(files), batch_size):

            batch = files[i:i + batch_size]
            imgs, masks = [], []

            for img_file in batch:

                img_path = os.path.join(img_dir, img_file)
                mask_file = os.path.splitext(img_file)[0] + ".png"
                mask_path = os.path.join(mask_dir, mask_file)

                if not os.path.exists(mask_path):
                    continue

                img, mask = load_image_mask(img_path, mask_path)

                if transforms:
                    augmented = transforms(image=img, mask=mask)
                    img = augmented['image']
                    mask = augmented['mask']


                imgs.append(img)
                masks.append(mask)

            if len(imgs) > 0:
                yield np.array(imgs), np.array(masks)


train_gen = subset_generator(
    TRAIN_IMG,
    TRAIN_MASK,
    train_files,
    BATCH_SIZE,
    transforms=train_transforms
)

val_gen = subset_generator(
    TRAIN_IMG,
    TRAIN_MASK,
    val_files,
    BATCH_SIZE
)


# Fit Model
model.fit(
    train_gen,
    steps_per_epoch=len(train_files) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(val_files) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[lr_scheduler, early_stopping]
)

Train samples: 3146
Val samples: 787
Epoch 1/25


/tmp/ipykernel_14933/4144839830.py:34: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
/tmp/ipykernel_14933/4144839830.py:36: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5),
/tmp/ipykernel_14933/4144839830.py:44: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.2),
2026-02-17 15:38:34.287170: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f23f80032b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-17 15:38:34.287199: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2026-02-17 15:38:34.415116: I tenso

393/393 ━━━━━━━━━━━━━━━━━━━━ 190s 388ms/step - accuracy: 0.9743 - dice_coefficient: 0.1572 - loss: 0.9729 - val_accuracy: 0.9728 - val_dice_coefficient: 0.2088 - val_loss: 0.8999 - learning_rate: 1.0000e-04
Epoch 2/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 164s 382ms/step - accuracy: 0.9865 - dice_coefficient: 0.3920 - loss: 0.6683 - val_accuracy: 0.9902 - val_dice_coefficient: 0.4950 - val_loss: 0.5525 - learning_rate: 1.0000e-04
Epoch 3/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 155s 394ms/step - accuracy: 0.9888 - dice_coefficient: 0.5306 - loss: 0.5203 - val_accuracy: 0.9891 - val_dice_coefficient: 0.6042 - val_loss: 0.4435 - learning_rate: 1.0000e-04
Epoch 4/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 149s 380ms/step - accuracy: 0.9899 - dice_coefficient: 0.6012 - loss: 0.4484 - val_accuracy: 0.9900 - val_dice_coefficient: 0.6457 - val_loss: 0.4036 - learning_rate: 1.0000e-04
Epoch 5/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 149s 380ms/step - accuracy: 0.9902 - dice_coefficient: 0.6254 - loss: 0.4241 - val_accuracy: 0.9920

In [10]:
def predict_and_save(model, img_dir, mask_dir):

    os.makedirs(PRED_MASK_DIR, exist_ok=True)
    os.makedirs(OVERLAY_DIR, exist_ok=True)

    img_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    for file in img_files:

        # image path
        img_path = os.path.join(img_dir, file)

        # mask name
        mask_file = os.path.splitext(file)[0] + ".png"
        mask_path = os.path.join(mask_dir, mask_file)

        # skip if mask missing 
        if not os.path.exists(mask_path):
            print(f"Skipping (mask not found): {file}")
            continue

        # load image & mask 
        img, true_mask = load_image_mask(img_path, mask_path)

        # prediction 
        pred = model.predict(img[np.newaxis, ...], verbose=0)[0]
        pred_mask = (pred > 0.5).astype(np.uint8) * 255

        # save predicted mask 
        cv2.imwrite(
            os.path.join(PRED_MASK_DIR, mask_file),
            pred_mask.squeeze()
        )

        # overlay 
        img_vis = cv2.imread(img_path)
        img_vis = cv2.resize(img_vis, (IMG_SIZE, IMG_SIZE))

        overlay = img_vis.copy()
        overlay[pred_mask.squeeze() == 255] = [0, 0, 255]  # red tumor

        combined = cv2.addWeighted(img_vis, 0.7, overlay, 0.3, 0)

        cv2.imwrite(
            os.path.join(OVERLAY_DIR, file),
            combined
        )

    print("Predictions, masks & overlays saved successfully.")


In [11]:
predict_and_save(model, TEST_IMG, TEST_MASK)

Predictions, masks & overlays saved successfully.


In [12]:
model.save("brisc2025_unet_latest.keras")

In [13]:
from tensorflow.keras.models import load_model

model = load_model(
    "brisc2025_unet_latest.keras",
    custom_objects={
        "bce_dice_loss": bce_dice_loss,
        "dice_coefficient": dice_coefficient
    }
)

In [14]:
from tqdm import tqdm
import os
import numpy as np

# Metric helpers (NumPy)

def dice_np(y_true, y_pred, smooth=1e-6):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    intersection = np.sum(y_true * y_pred)

    return (2. * intersection + smooth) / (
        np.sum(y_true) + np.sum(y_pred) + smooth
    )


def iou_np(y_true, y_pred, smooth=1e-6):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection

    return (intersection + smooth) / (union + smooth)


def precision_recall_np(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)

    return precision, recall


def accuracy_np(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    correct = np.sum(y_true == y_pred)
    total = len(y_true)

    return correct / total


# Evaluate on test set

def evaluate_test_set(model, img_dir, mask_dir):

    image_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    dice_scores = []
    iou_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    print(" Evaluating test set...")

    for file in tqdm(image_files):

        img_path = os.path.join(img_dir, file)

        mask_path = os.path.join(
            mask_dir,
            os.path.splitext(file)[0] + ".png"
        )

        if not os.path.exists(mask_path):
            continue

        img, gt_mask = load_image_mask(img_path, mask_path)

        pred = model.predict(img[np.newaxis, ...], verbose=0)[0]
        pred_bin = (pred > 0.5).astype(np.uint8)

        gt_bin = gt_mask.astype(np.uint8)

        dice_scores.append(dice_np(gt_bin, pred_bin))
        iou_scores.append(iou_np(gt_bin, pred_bin))

        p, r = precision_recall_np(gt_bin, pred_bin)
        precision_scores.append(p)
        recall_scores.append(r)

        accuracy_scores.append(
            accuracy_np(gt_bin, pred_bin)
        )

    print("\n===== TEST SET RESULTS =====")
    print(f"Mean Dice     : {np.mean(dice_scores):.4f}")
    print(f"Mean IoU      : {np.mean(iou_scores):.4f}")
    print(f"Mean Precision: {np.mean(precision_scores):.4f}")
    print(f"Mean Recall   : {np.mean(recall_scores):.4f}")
    print(f"Mean Accuracy : {np.mean(accuracy_scores):.4f}")

    return {
        "dice": dice_scores,
        "iou": iou_scores,
        "precision": precision_scores,
        "recall": recall_scores,
        "accuracy": accuracy_scores,
    }


# RUN EVALUATION
results = evaluate_test_set(
    model,
    TEST_IMG,
    TEST_MASK
)


 Evaluating test set...


100%|████████████████████████████████████████████████████████████████████████████████████████| 860/860 [00:53<00:00, 16.13it/s]


===== TEST SET RESULTS =====
Mean Dice     : 0.7712
Mean IoU      : 0.6890
Mean Precision: 0.7972
Mean Recall   : 0.8067
Mean Accuracy : 0.9936
